In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re
import spacy
from pandarallel import pandarallel

!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_lg

In [2]:
nlp = spacy.load("en_core_web_lg")
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
df = pd.read_pickle("p:/uni/nlp/WashingtonPost_v4/data/v5/RAWData.pkl")

In [4]:
df.Kategorie = df.Kategorie.replace("\"$|\'$", "",regex=True)
df.Titel = df.Titel.replace("\"$|\'$", "",regex=True)
df.Datum = df.Datum.replace("\"$|\'$", "",regex=True) 

In [5]:
df = df.set_index('id')
df=df[:10]

In [6]:
df.content = df.content.replace("\<.*?\>","",regex=True)

In [7]:
df["tokens"] = df.parallel_apply(lambda row: [a.text for a in list(nlp(str(row["content"])))], axis=1)

In [52]:
df

,Kategorie,Titel,Autor,Datum,content
id,,,,,
0,Education,Some students,Emma Brown,2012-01-01T22:13:59Z,Nevermind that it’s a federal holiday. And nev...
1,The Post's View,Medical experiments on humans,Editorial Board,2012-01-02T00:39:34Z,"IN 2010, THE FEDERAL government funded 55,000 ..."
2,Politics,Iowa caucuses: One day out.,Karen Tumulty; David A. Fahrenthold,2012-01-03T01:37:00Z,DES MOINES — One candidate made an appearance ...
3,Technology,Rumor roundup: ‘iPad 3’ displays,Hayley Tsukayama,2012-01-03T13:58:00Z,With the new year comes a fresh burst of excit...
4,Technology,Nintendo 3DS hits 4M in sales,Hayley Tsukayama,2012-01-03T15:42:08Z,"Nintendo’s newest piece of hardware, the Ninte..."
...,...,...,...,...,...
12328,Letters to the Editor,Trickle-down is the zombie economic idea that ...,Letters to the Editor,2020-12-30T21:25:51Z,The Dec. 24 news analysis “\u2009‘Trickle-down...
12329,College Sports,Wisconsin quarterback’s untimely fumble shatte...,Cindy Boren,None,A funny thing happened to the Wisconsin Badger...
12330,Business,The stock market is ending 2020 at record highs,Hamza Shaban; Heather Long,None,The U.S. stock market ended 2020 at all-time h...


In [ ]:
all_stopwords = nlp.Defaults.stop_words
def remove_stopwords_from_list(list_in):
    return [a for a in list_in if a not in all_stopwords]

In [ ]:
df["no_stop"] = data.parallel_apply(lambda row: remove_stopwords_from_list(row["tokens"]), axis=1)
data["lemma"] = data.parallel_apply(lambda row: [t.lemma_ for t in nlp(row["content"])], axis=1)